# NICT Badge & Wallet System — Code Examples

Welcome! This notebook provides practical code examples for using the NICT Badge & Wallet System. You'll learn how to:
- Work with database models (see `nictbw/models`)
- Interact with the blockchain client (see `nictbw/blockchain`)

## Before You Start
- Set up your Python environment as described in the [Quick Start](./README.md#quick-start) section of the README.
- Initialize and seed the database (see [README.md](./README.md)).

---

## Working with the Database

Most database operations use **SQLAlchemy ORM 2.0 style** with a sessionmaker and context manager. This makes transactions safe and easy.

### Step 1: Setup
Create an engine and sessionmaker:

In [ ]:
from nictbw.db.engine import make_engine
from sqlalchemy.orm import sessionmaker

engine = make_engine()  # This uses the `DB_URL` env variable
Session = sessionmaker(engine)

### Step 2: Use the Session

Put your database code inside a `with` block:

In [ ]:
with Session.begin() as session:
    # Your DB code goes here
    # ...

    pass  # Adding this just to avoid syntax error. Remove in actual code.

That's it! The session will **automatically commit** if no errors occur, or **roll back** if there is an exception.

- You do NOT need to call `session.commit()` or `session.rollback()` manually.
- Call `session.flush()` inside the block if you want to send changes to the database before the block ends.

---

## Examples

### Example 1: Create and Query a User

Let's create a new user and see how to query them from the database.

In [ ]:
from nictbw.models import User

with Session.begin() as session:
    # Create a new User
    user = User(
        in_app_id="example_user_01",
        paymail="01@example.com",
        nickname="Bob",
        on_chain_id="test1"
    )

    # Add the user to the DB
    session.add(user)

    # As the session ends, the new user is committed to the DB

**Note:**
- If you run the cell above multiple times, you may get a `UNIQUE constraint failed` error. This means the `in_app_id` and `wallet` fields must be unique for each user.
- To avoid this, change the values of `in_app_id` and `wallet` each time you run the cell.
- The same rule applies to other models with unique fields.

Now, let's check if the user was saved in the database by querying for them:

In [ ]:
with Session.begin() as session:
    # Query the user by in_app_id.
    user = User.get_by_in_app_id(session, "example_user_01")

    if user:
        print("It works!")
        print(user)
        print(f"This user is created at {user.created_at}.")
    else:
        print("User not found :(")

Here we use the `get_by_in_app_id` class method from the `User` model. There are other useful methods like `get_by_wallet`, `get_by_on_chain_id`, etc. Check the model definitions for more!

### Example 2: Create an NFT, Mint it On-Chain, and Assign to a User

This example shows how to:
1. Create an NFT in the database
2. Mint it on the blockchain
3. Assign it to a user

In [ ]:
from nictbw.models import NFT, Admin, User
from nictbw.blockchain.api import ChainClient

with Session.begin() as session:
    # Step 1: Get the admin and user from the database
    admin = Admin.get_by_paymail(session, "admin@example.com")
    if not admin:
        raise ValueError("Admin with paymail 'admin@example.com' not found")

    user = User.get_by_in_app_id(session, "example_user_01")
    if not user:
        raise ValueError("User with in_app_id 'example_user_01' not found")

    # Step 2: Create the NFT in the database
    nft = NFT(
        prefix="CoolPrefix",
        shared_key="some_shared_key_123",
        name="Sample Badge",
        nft_type="badge",
        created_by_admin_id=admin.id,  # Admin's id is supplied here
        description="Sample badge NFT",
    )

    session.add(nft)
    session.flush()  # Flush the session (i.e. write to the DB) to ensure the NFT gets its `id` (primary key).
    print(f"NFT created with id: {nft.id}")

    # Step 3: Mint the NFT on-chain
    chain_client = ChainClient()
    response = nft.mint_on_chain(
        session,
        chain_client,
        app="Sample App",
        # recipient_paymail=user.paymail  # Uncomment if the user has a real paymail
    )
    print(f"NFT minted on-chain. Transaction ID: {response['transaction_id']}")
    print(f"NFT on-chain information: {response['nft_information']}")

    # Step 4: Assign the NFT to the user in the database
    user.issue_nft_dbwise(session, nft)

To verify the NFT was created and assigned, query the database:

You can fetch a user's NFT ownerships directly using relationships in the `User` model:

In [ ]:
with Session.begin() as session:
    # Fetch the user from the database
    user = User.get_by_in_app_id(session, "example_user_01")
    if not user:
        raise ValueError("User with in_app_id 'example_user_01' not found")

    # Print all NFT ownership records for this user
    print(user.ownerships)  # List of ownership objects
    # Print all NFTs assigned to this user
    print(user.nfts)  # List of NFT objects

---

### Example 3: Get User and NFT Info from the Blockchain

Use the blockchain client to fetch user and NFT information directly from the chain.

In [ ]:
from nictbw.blockchain.api import ChainClient

# Create a blockchain client using environment variables
# By default, this logs in as the admin user (if `.env` vars are set)
chain_client = ChainClient()
chain_client.info

In [ ]:
# Get all NFTs for a user from the blockchain using their on-chain ID
# This uses admin credentials to access blockchain data
chain_client.get_user_nfts("test1")

# Alternatively, get sorted NFTs
# chain_client.get_sorted_user_nfts("test1")

In [ ]:
# Fetch detailed info for a specific NFT from the blockchain using its origin value
# Note: As of blockchain API v1, this only returns the uploaded image data for the NFT.
# For full metadata support, future API updates may be required.
chain_client.get_nft_info("some_origin_value")  # Replace with the actual origin value

---

*Work in Progress*: More examples and advanced usage will be added soon.

In [ ]:
# WIP: Sync NFTs from blockchain to DB for a user
# Need to handle the case where on-chain NFTs are not in the DB yet

# For a user with `on_chain_id` "test1", update the DB with their NFTs from the blockchain:

# with Session.begin() as session:
#     user = User.get_by_on_chain_id(session, "test1")
#     if not user:
#         raise ValueError("User with on_chain_id 'test1' not found")
#     # Sync NFTs from blockchain to DB
#     user.sync_nfts_from_chain(session, chain_client)
#     print(user.nfts)  # Print updated NFT list